# Predicting Income Status

The objective of this case study is to fit and compare three different binary classifiers to predict whether an individual earns more than USD 50,000 (50K) or less in a year using the 1994 US Census Data sourced from the UCI Machine Learning Repository (Lichman, 2013). The descriptive features include 4 numeric and 7 nominal categorical features. The target feature has two classes defined as "<=50K" and ">50K" respectively. The full dataset contains about 45K observations.

This report is organized as follows:
- [Section 2 (Overview)](#2) outlines our methodology. 
- [Section 3 (Data Preparation)](#3) summarizes the data preparation process and our model evaluation strategy. 
- [Section 4 (Hyperparameter Tuning)](#4) describes the hyperparameter tuning process for each classification algorithm.
- [Section 5 (Performance Comparison)](#5) presents model performance comparison results.
- [Section 6 (Limitations)](#6) discusses a limitations of our approach and possible solutions. 
- [Section 7 (Summary)](#7) provides a brief summary of our work in this project.

Compiled from a Jupyter Notebook, this report contains both narratives and the Python code used throughout the project.

# Overview <a class="anchor" id="2"></a> 

### Methodology

We build the following binary classifiers to predict the target feature:

* K-Nearest Neighbors (KNN),
* Decision trees (DT), and
* Naive Bayes (NB).

Our modeling strategy begins by transforming the full dataset cleaned from Phase I. This transformation includes encoding categorical descriptive features as numerical and then scaling of the descriptive features. We first randomly sample 20K rows from the full dataset and then split this sample into training and test sets with a 70:30 ratio. This way, our training data has 14K rows and test data has 6K rows.

Before fitting a particular model, we select the best features using the powerful Random Forest Importance method inside a pipeline. We consider 10, 20, and the full set of features (with 41 features) after encoding of categorical features.

Using feature selection together with hyperparameter search inside a single pipeline, we conduct a 5-fold stratified cross-validation to fine-tune hyperparameters of each classifier using area under curve (AUC) as the performance metric. We build each model using parallel processing with "-2" cores. Since the target has more individuals earning less than USD 50K in 1994 (unbalanced target class issue), stratification is crucial to ensure that each validation set has the same proportion of classes as in the original dataset. We also examine sensitivity of each model with respect to its hyperparameters during the search.

Once the best model is identified for each of the three classifier types using a hyperparameter search on the training data, we conduct a 10-fold cross-validation on the test data and perform a paired t-test to see if any performance difference is statistically significant. In addition, we compare the classifiers with respect to their recall scores and confusion matrices on the test data.

# Data Preparation <a class="anchor" id="3"></a> 

## Loading Dataset

We load the dataset from the Cloud. Below we set the seed to a particular value at the beginning of this notebook so that our results can be repeated later on.

In [109]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import io
import requests
import os, ssl

In [110]:
heart_disease_url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/processed.cleveland.data'

heart_disease = pd.read_csv(heart_disease_url, sep=',', header = None, names =['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg',
                   'thalach', 'exang', 'oldpeak', 'slope', 'ca', 'thal', 'num'], encoding='utf-8', na_values = '?', keep_default_na=False)

print(heart_disease.shape)

heart_disease.columns.values

(303, 14)


array(['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg',
       'thalach', 'exang', 'oldpeak', 'slope', 'ca', 'thal', 'num'],
      dtype=object)

The full data has 303 observations. It has 14 descriptive features and "num" is the target feature. 

## Checking for Missing Values

Let's make sure we do not have any missing values.

In [111]:
heart_disease.isna().sum()

age         0
sex         0
cp          0
trestbps    0
chol        0
fbs         0
restecg     0
thalach     0
exang       0
oldpeak     0
slope       0
ca          4
thal        2
num         0
dtype: int64

In [112]:
heart_disease.dtypes

age         float64
sex         float64
cp          float64
trestbps    float64
chol        float64
fbs         float64
restecg     float64
thalach     float64
exang       float64
oldpeak     float64
slope       float64
ca          float64
thal        float64
num           int64
dtype: object

In [113]:
heart_disease['age'] = heart_disease['age'].astype(int)

heart_disease['sex'] = heart_disease['sex'].replace({0: 'Female', 1: 'Male'})

heart_disease['cp'] = heart_disease['cp'].replace({1: 'Typical Angina', 2: 'Atypical Angina', 3: 'Non-Anginal pain', 
                                                   4: 'Asymptomatic'})

heart_disease['trestbps'] = heart_disease['trestbps'].astype(int)

heart_disease['chol'] = heart_disease['chol'].astype(int)

heart_disease['fbs'] = heart_disease['fbs'].replace({0: 'False', 1: 'True'})

heart_disease['restecg'] = heart_disease['restecg'].replace({0: 'Normal', 1: 'Abnormality', 2: 'Hypertrophy'})

heart_disease['thalach'] = heart_disease['thalach'].astype(int)

heart_disease['exang'] = heart_disease['exang'].replace({0: 'No', 1: 'Yes'})

heart_disease['slope'] = heart_disease['slope'].replace({1: 'Up', 2: 'Flat', 3: 'Down'})

heart_disease['ca'] = heart_disease['ca'].replace({0: 'No vessel', 1: 'One vessel', 2: 'Two vessel', 3: 'Three vessel'})

heart_disease['thal'] = heart_disease['thal'].replace({3: 'Normal', 6: 'Fixed Defect', 7: 'Reversable Defect'}) 


Let's have a look at 5 randomly selected rows in this raw dataset.

In [114]:
heart_disease.sample(n=5, random_state=999)

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,num
256,67,Female,Asymptomatic,106,223,False,Normal,142,No,0.3,Up,Two vessel,Normal,0
278,57,Male,Atypical Angina,154,232,False,Hypertrophy,164,No,0.0,Up,One vessel,Normal,1
123,55,Male,Asymptomatic,140,217,False,Normal,111,Yes,5.6,Down,No vessel,Reversable Defect,3
128,44,Male,Atypical Angina,120,220,False,Normal,170,No,0.0,Up,No vessel,Normal,0
156,51,Male,Asymptomatic,140,299,False,Normal,173,Yes,1.6,Up,No vessel,Reversable Defect,1


In [115]:
heart_disease.isna().sum()

age         0
sex         0
cp          0
trestbps    0
chol        0
fbs         0
restecg     0
thalach     0
exang       0
oldpeak     0
slope       0
ca          4
thal        2
num         0
dtype: int64

## Summary Statistics

The summary statistics for the full data are shown below.

In [116]:
heart_disease.describe(include='all')

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,num
count,303.000000,303,303,303.000000,303.000000,303,303,303.000000,303,303.000000,303,299,301,303.000000
unique,NaN,2,4,NaN,NaN,2,3,NaN,2,NaN,3,4,3,NaN
top,NaN,Male,Asymptomatic,NaN,NaN,False,Normal,NaN,No,NaN,Up,No vessel,Normal,NaN
freq,NaN,206,144,NaN,NaN,258,151,NaN,204,NaN,142,176,166,NaN
mean,54.438944,NaN,NaN,131.689769,246.693069,NaN,NaN,149.607261,NaN,1.039604,NaN,NaN,NaN,0.937294
std,9.038662,NaN,NaN,17.599748,51.776918,NaN,NaN,22.875003,NaN,1.161075,NaN,NaN,NaN,1.228536
min,29.000000,NaN,NaN,94.000000,126.000000,NaN,NaN,71.000000,NaN,0.000000,NaN,NaN,NaN,0.000000
25%,48.000000,NaN,NaN,120.000000,211.000000,NaN,NaN,133.500000,NaN,0.000000,NaN,NaN,NaN,0.000000
50%,56.000000,NaN,NaN,130.000000,241.000000,NaN,NaN,153.000000,NaN,0.800000,NaN,NaN,NaN,0.000000
75%,61.000000,NaN,NaN,140.000000,275.000000,NaN,NaN,166.000000,NaN,1.600000,NaN,NaN,NaN,2.000000


## Encoding Categorical Features

Prior to modeling, it is essential to encode all categorical features (both the target feature and the descriptive features) into a set of numerical features.

### Encoding the Target Feature
We remove the "income_status" feature from the full dataset and call it "target". The rest of the features are the descriptive features which we call "Data".

In [117]:
Data = heart_disease.drop(columns='num')
target = heart_disease['num']
target.value_counts()

0    164
1     55
2     36
3     35
4     13
Name: num, dtype: int64

Let's encode the target feature so that the positive class is ">50K" and it is encoded as "1".

In [118]:
target = target.replace({0: 0, 1: 1, 2: 1, 3: 1, 4: 1})
target.value_counts()

0    164
1    139
Name: num, dtype: int64

As a side note, we observe that the classes are not quite balanced. 

### Encoding Categorical Descriptive Features

Since all of the descriptive features appear to be nominal, we perform one-hot-encoding. Furthermore, since we plan on conducting feature selection, we define $q$ dummy variables for a categorical descriptive variable with $q$ levels. The exception here is that when a categorical descriptive feature has only two levels, we define a single dummy variable. Let's extract the list of categorical descriptive features.

In [119]:
categorical_cols = Data.columns[Data.dtypes==object].tolist()

Before any transformation, the categorical features are as follows.

In [120]:
categorical_cols

['sex', 'cp', 'fbs', 'restecg', 'exang', 'slope', 'ca', 'thal']

In [121]:
heart_disease.dtypes

age           int32
sex          object
cp           object
trestbps      int32
chol          int32
fbs          object
restecg      object
thalach       int32
exang        object
oldpeak     float64
slope        object
ca           object
thal         object
num           int64
dtype: object

The coding operation is shown below. For each two-level categorical variable, we set the `drop_first` option to `True` to encode the variable into a single column of 0 or 1. Next, we apply the `get_dummies()` function for the regular one-hot encoding for categorical features with more than 2 levels.

In [122]:
for col in categorical_cols:
    n = len(Data[col].unique())
    if (n == 2):
        Data[col] = pd.get_dummies(Data[col], drop_first=True)
   
# use one-hot-encoding for categorical features with >2 levels
Data = pd.get_dummies(Data)

After encoding, the feature set has the following columns.

In [123]:
Data.columns

Index([u'age', u'sex', u'trestbps', u'chol', u'fbs', u'thalach', u'exang',
       u'oldpeak', u'cp_Asymptomatic', u'cp_Atypical Angina',
       u'cp_Non-Anginal pain', u'cp_Typical Angina', u'restecg_Abnormality',
       u'restecg_Hypertrophy', u'restecg_Normal', u'slope_Down', u'slope_Flat',
       u'slope_Up', u'ca_No vessel', u'ca_One vessel', u'ca_Three vessel',
       u'ca_Two vessel', u'thal_Fixed Defect', u'thal_Normal',
       u'thal_Reversable Defect'],
      dtype='object')

In [124]:
Data.sample(5, random_state=999)

,age,sex,trestbps,chol,fbs,thalach,exang,oldpeak,cp_Asymptomatic,cp_Atypical Angina,...,slope_Down,slope_Flat,slope_Up,ca_No vessel,ca_One vessel,ca_Three vessel,ca_Two vessel,thal_Fixed Defect,thal_Normal,thal_Reversable Defect
256,67,0,106,223,0,142,0,0.3,1,0,...,0,0,1,0,0,0,1,0,1,0
278,57,1,154,232,0,164,0,0.0,0,1,...,0,0,1,0,1,0,0,0,1,0
123,55,1,140,217,0,111,1,5.6,1,0,...,1,0,0,1,0,0,0,0,0,1
128,44,1,120,220,0,170,0,0.0,0,1,...,0,0,1,1,0,0,0,0,1,0
156,51,1,140,299,0,173,1,1.6,1,0,...,0,0,1,1,0,0,0,0,0,1


## Scaling of Features

After encoding all the categorical features, we perform a min-max scaling of the descriptive features. But first we make a copy of the Data to keep track of column names.

In [125]:
from sklearn import preprocessing

Data_df = Data.copy()

Data_scaler = preprocessing.MinMaxScaler()
Data_scaler.fit(Data)
Data = Data_scaler.fit_transform(Data)

Let's have another look at the descriptive features after scaling. Pay attention that the output of the scaler is a `NumPy` array, so all the column names are lost. That's why we kept a copy of Data before scaling so that we can recover the column names below. We observe below that binary features are still kept as binary after the min-max scaling.

In [126]:
pd.DataFrame(Data, columns=Data_df.columns).sample(5, random_state=999)

,age,sex,trestbps,chol,fbs,thalach,exang,oldpeak,cp_Asymptomatic,cp_Atypical Angina,...,slope_Down,slope_Flat,slope_Up,ca_No vessel,ca_One vessel,ca_Three vessel,ca_Two vessel,thal_Fixed Defect,thal_Normal,thal_Reversable Defect
256,0.791667,0.0,0.113208,0.221461,0.0,0.541985,0.0,0.048387,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
278,0.583333,1.0,0.566038,0.242009,0.0,0.709924,0.0,0.000000,0.0,1.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
123,0.541667,1.0,0.433962,0.207763,0.0,0.305344,1.0,0.903226,1.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
128,0.312500,1.0,0.245283,0.214612,0.0,0.755725,0.0,0.000000,0.0,1.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
156,0.458333,1.0,0.433962,0.394977,0.0,0.778626,1.0,0.258065,1.0,0.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0


## Feature Selection & Ranking

Let's have a look at the most important 10 features as selected by Random Forest Importance (RFI) in the full dataset. This is for a quick ranking of the most relevant 10 features to gain some insight into the problem at hand. During the hyperparameter tuning phase, we will include RFI as part of the pipeline and we will search over 10, 20, and the full set of 41 features to determine which number of features works best with each classifier.

In [127]:
from sklearn.ensemble import RandomForestClassifier

num_features = 10
model_rfi = RandomForestClassifier(n_estimators=100)
model_rfi.fit(Data, target)
fs_indices_rfi = np.argsort(model_rfi.feature_importances_)[::-1][0:num_features]

best_features_rfi = Data_df.columns[fs_indices_rfi].values
best_features_rfi

array(['thalach', 'cp_Asymptomatic', 'oldpeak', 'age', 'thal_Normal',
       'thal_Reversable Defect', 'ca_No vessel', 'trestbps', 'chol',
       'exang'], dtype=object)

In [128]:
feature_importances_rfi = model_rfi.feature_importances_[fs_indices_rfi]
feature_importances_rfi

array([0.10015804, 0.09736129, 0.09268504, 0.08654885, 0.07805709,
       0.07581099, 0.07207303, 0.06529294, 0.06427253, 0.05118647])

Let's visualize these importances.

In [130]:
import altair as alt

def plot_imp(best_features, scores, method_name, color):
    
    df = pd.DataFrame({'features': best_features, 
                       'importances': scores})
    
    chart = alt.Chart(df, 
                      width=500, 
                      title=method_name + ' Feature Importances'
                     ).mark_bar(opacity=0.85, 
                                color=color).encode(
        alt.X('features', title='Feature', sort=None, axis=alt.AxisConfig(labelAngle=45)),
        alt.Y('importances', title='Importance')
    )
    
    return chart

ImportError: No module named altair

In [131]:
plot_imp(best_features_rfi, feature_importances_rfi, 'Random Forest', 'blue')

NameError: name 'plot_imp' is not defined

We observe that the most important feature is age, followed by capital, education, and hours per week.

## Data Sampling & Train-Test Splitting

The original dataset has more than 45K rows, which is a lot. So, we would like to work with a small sample here with 20K rows. Thus, we will do the following:
- Randomly select 20K rows from the full dataset.
- Split this sample into train and test partitions with a 70:30 ratio using stratification.

Pay attention here that we use `values` attribute to convert `Pandas` data frames to a `NumPy` array. You have to make absolutely sure that you **NEVER** pass `Pandas` data frames to `Scikit-Learn` functions!!! Sometimes it will work. But sometimes you will end up getting strange errors such as "invalid key" etc. Remember, `Scikit-Learn` works with `NumPy` arrays, not `Pandas` data frames.

In [132]:
n_samples = 297

Data_sample = pd.DataFrame(Data).sample(n=n_samples, random_state=8).values
target_sample = pd.DataFrame(target).sample(n=n_samples, random_state=8).values

print(Data_sample.shape)
print(target_sample.shape)

(297L, 25L)
(297L, 1L)


In [133]:
from sklearn.model_selection import train_test_split

Data_sample_train, Data_sample_test, \
target_sample_train, target_sample_test = train_test_split(Data_sample, target_sample, 
                                                    test_size = 0.3, random_state=999,
                                                    stratify = target_sample)

print(Data_sample_train.shape)
print(Data_sample_test.shape)

(207L, 25L)
(90L, 25L)


## Model Evaluation Strategy

So, we will train and tune our models on 14K rows of training data and we will test them on 6K rows of test data. 

For each model, we will use 5-fold stratified cross-validation evaluation method (without any repetitions for shorter run times) for hyperparameter tuning.

In [134]:
from sklearn.model_selection import StratifiedKFold, GridSearchCV

cv_method = StratifiedKFold(n_splits=5, random_state=999)

# Hyperparameter Tuning <a class="anchor" id="4"></a> 

## K-Nearest Neighbors (KNN)

Using `Pipeline`, we stack feature selection and grid search for KNN hyperparameter tuning via cross-validation. We will use the same `Pipeline` methodology for NB and DT.

The KNN hyperparameters are as follows:

* number of neighbors (`n_neighbors`) and
* the distance metric `p`.

For feature selection, we use the powerful Random Forest Importance (RFI) method with 100 estimators. A trick here is that we need a bit of coding so that we can make RFI feature selection as part of the pipeline. For this reason, we define the custom `RFIFeatureSelector()` class
below to pass in RFI as a "step" to the pipeline.

In [135]:
from sklearn.base import BaseEstimator, TransformerMixin

# custom function for RFI feature selection inside a pipeline
# here we use n_estimators=100
class RFIFeatureSelector(BaseEstimator, TransformerMixin):
    
    # class constructor 
    # make sure class attributes end with a "_"
    # per scikit-learn convention to avoid errors
    def __init__(self, n_features_=10):
        self.n_features_ = n_features_
        self.fs_indices_ = None

    # override the fit function
    def fit(self, X, y):
        from sklearn.ensemble import RandomForestClassifier
        from numpy import argsort
        model_rfi = RandomForestClassifier(n_estimators=100)
        model_rfi.fit(X, y)
        self.fs_indices_ = argsort(model_rfi.feature_importances_)[::-1][0:self.n_features_] 
        return self 
    
    # override the transform function
    def transform(self, X, y=None):
        return X[:, self.fs_indices_]


In [136]:
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier

pipe_KNN = Pipeline(steps=[('rfi_fs', RFIFeatureSelector()), 
                           ('knn', KNeighborsClassifier())])

params_pipe_KNN = {'rfi_fs__n_features_': [10, 20, Data.shape[1]],
                   'knn__n_neighbors': [1, 10, 20, 40, 60, 100],
                   'knn__p': [1, 2]}

gs_pipe_KNN = GridSearchCV(estimator=pipe_KNN, 
                           param_grid=params_pipe_KNN, 
                           cv=cv_method,
                           refit=True,
                           n_jobs=-2,
                           scoring='roc_auc',
                           verbose=1) 

In [137]:
gs_pipe_KNN.fit(Data_sample_train, target_sample_train);

Fitting 5 folds for each of 36 candidates, totalling 180 fits


[Parallel(n_jobs=-2)]: Using backend LokyBackend with 7 concurrent workers.
[Parallel(n_jobs=-2)]: Done  36 tasks      | elapsed:   13.1s
[Parallel(n_jobs=-2)]: Done 180 out of 180 | elapsed:   25.3s finished


In [138]:
gs_pipe_KNN.best_params_

{'knn__n_neighbors': 60, 'knn__p': 2, 'rfi_fs__n_features_': 25L}

In [139]:
gs_pipe_KNN.best_score_

0.9212053878753709

We observe that the optimal KNN model has a mean AUC score of 0.921 . The best performing KNN selected 10 features with 40 nearest neighbors and $p=2$.

Even though these are the best values, let's have a look at the other combinations to see if the difference is rather significant or not. For this, we will make use of the function below to format the grid search outputs as a `Pandas` data frame.

In [143]:
# custom function to format the search results as a Pandas data frame
def get_search_results(gs):

    def model_result(scores, params):
        scores = {'mean_score': np.mean(scores),
             'std_score': np.std(scores),
             'min_score': np.min(scores),
             'max_score': np.max(scores)}
        return pd.Series({params,scores})

    models = []
    scores = []

    for i in range(gs.n_splits_):
        key = f"split{i}_best_score"
        r = gs.cv_results_[key]        
        scores.append(r.reshape(-1,1))

    all_scores = np.hstack(scores)
    for p, s in zip(gs.cv_results_['params'], all_scores):
        models.append((model_result(s, p)))

    pipe_results = pd.concat(models, axis=1).T.sort_values(['mean_score'], ascending=False)

    columns_first = ['mean_score', 'std_score', 'max_score', 'min_score']
    columns = columns_first + [c for c in pipe_results.columns if c not in columns_first]

    return pipe_results[columns]

SyntaxError: invalid syntax (<ipython-input-143-61d19706e644>, line 15)

In [85]:
results_KNN = get_search_results(gs_pipe_KNN)
results_KNN.head()

KeyError: 'fsplit{i}_test_score'

We observe that the difference between the hyperparameter combinations is not really much when conditioned on the number of features selected. Let's visualize the results of the grid search corresponding to 10 selected features.

In [ ]:
import altair as alt

results_KNN_10_features = results_KNN[results_KNN['rfi_fs__n_features_'] == 10.0]

alt.Chart(results_KNN_10_features, 
          title='KNN Performance Comparison with 10 Features'
         ).mark_line(point=True).encode(
    alt.X('knn__n_neighbors', title='Number of Neighbors'),
    alt.Y('mean_score', title='AUC Score', scale=alt.Scale(zero=False)),
    alt.Color('knn__p:N', title='p')
)

## (Gaussian) Naive Bayes (NB)

We implement a Gaussian Naive Bayes model.  We optimize `var_smoothing` (a variant of Laplace smoothing) as we do not have any prior information about our dataset. By default, the `var_smoothing` parameter's value is $10^{-9}$ . We conduct the grid search in the `logspace` (over the powers of 10) sourced from `NumPy`. We start with 10 and end with $10^{-3}$ with 200 different values, but we perform a random search over only 20 different values (for shorter run times). Since NB requires each descriptive feature to follow a Gaussian distribution, we first perform a power transformation on the input data before model fitting.

In [86]:
from sklearn.preprocessing import PowerTransformer
Data_sample_train_transformed = PowerTransformer().fit_transform(Data_sample_train)

In [87]:
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import RandomizedSearchCV

pipe_NB = Pipeline([('rfi_fs', RFIFeatureSelector()), 
                     ('nb', GaussianNB())])

params_pipe_NB = {'rfi_fs__n_features_': [10, 20, Data.shape[1]],
                  'nb__var_smoothing': np.logspace(1,-3, num=200)}

n_iter_search = 20
gs_pipe_NB = RandomizedSearchCV(estimator=pipe_NB, 
                          param_distributions=params_pipe_NB, 
                          cv=cv_method,
                          refit=True,
                          n_jobs=-2,
                          scoring='roc_auc',
                          n_iter=n_iter_search,
                          verbose=1) 

gs_pipe_NB.fit(Data_sample_train_transformed, target_sample_train);

Fitting 5 folds for each of 20 candidates, totalling 100 fits


[Parallel(n_jobs=-2)]: Using backend LokyBackend with 7 concurrent workers.
[Parallel(n_jobs=-2)]: Done  36 tasks      | elapsed:    3.2s
[Parallel(n_jobs=-2)]: Done 100 out of 100 | elapsed:    8.3s finished


In [88]:
gs_pipe_NB.best_params_

{'nb__var_smoothing': 7.232633896483537, 'rfi_fs__n_features_': 25L}

In [89]:
gs_pipe_NB.best_score_

0.907568873492662

The optimal NB yiels an AUC score of 0.878 (with 10 features) - slightly higher than that of KNN. At this point, we cannot conclude NB outperforms KNN. For this conclusion, we will have to perform a paired t-test on the test data as discussed further below.

In [90]:
results_NB = get_search_results(gs_pipe_NB)
results_NB.head()

KeyError: 'fsplit{i}_test_score'

Let's visualize the search results.

In [ ]:
results_NB_10_features = results_NB[results_NB['rfi_fs__n_features_'] == 10.0]

alt.Chart(results_NB_10_features, 
          title='NB Performance Comparison with 10 Features'
         ).mark_line(point=True).encode(
    alt.X('nb__var_smoothing', title='Var. Smoothing'),
    alt.Y('mean_score', title='AUC Score', scale=alt.Scale(zero=False))
)

## Decision Trees (DT)

We build a DT using gini index to maximize information gain. We aim to determine the optimal combinations of maximum depth (`max_depth`) and minimum sample split (`min_samples_split`).

In [91]:
from sklearn.tree import DecisionTreeClassifier

pipe_DT = Pipeline([('rfi_fs', RFIFeatureSelector()),
                    ('dt', DecisionTreeClassifier(criterion='gini'))])

params_pipe_DT = {'rfi_fs__n_features_': [10, 20, Data.shape[1]],
                  'dt__max_depth': [3, 4, 5],
                  'dt__min_samples_split': [2, 5]}

gs_pipe_DT = GridSearchCV(estimator=pipe_DT, 
                          param_grid=params_pipe_DT, 
                          cv=cv_method,
                          refit=True,
                          n_jobs=-2,
                          scoring='roc_auc',
                          verbose=1) 

gs_pipe_DT.fit(Data_sample_train, target_sample_train);

Fitting 5 folds for each of 18 candidates, totalling 90 fits


[Parallel(n_jobs=-2)]: Using backend LokyBackend with 7 concurrent workers.
[Parallel(n_jobs=-2)]: Done  36 tasks      | elapsed:    3.4s
[Parallel(n_jobs=-2)]: Done  90 out of  90 | elapsed:    7.3s finished


In [92]:
gs_pipe_DT.best_params_

{'dt__max_depth': 3, 'dt__min_samples_split': 5, 'rfi_fs__n_features_': 20}

In [93]:
gs_pipe_DT.best_score_

0.7878199968041775

The best DT has a maximum depth of 5 and minimum split value of 2 samples with an AUC score of 0.881. A visualization of the search results is given below.

In [94]:
results_DT = get_search_results(gs_pipe_DT)

results_DT_10_features = results_DT[results_DT['rfi_fs__n_features_'] == 10.0]

alt.Chart(results_DT_10_features, 
          title='DT Performance Comparison with 10 Features'
         ).mark_line(point=True).encode(
    alt.X('dt__min_samples_split', title='Min Samples for Split'),
    alt.Y('mean_score', title='AUC Score', scale=alt.Scale(zero=False)),
    alt.Color('dt__max_depth:N', title='Max Depth')
)

KeyError: 'fsplit{i}_test_score'

## Further Fine Tuning

We notice that the optimal value of maximum depth hyperparameter is at the extreme end of its search space. Thus, we need to go beyond what we already tried to make sure that we are not missing out on even better values. For this reason, we try a new search as below.    

In [97]:
params_pipe_DT2 = {'rfi_fs__n_features_': [10],
                  'dt__max_depth': [5, 10, 15],
                  'dt__min_samples_split': [5, 50, 100, 150]}

gs_pipe_DT2 = GridSearchCV(estimator=pipe_DT, 
                          param_grid=params_pipe_DT2, 
                          cv=cv_method,
                          refit=True,
                          n_jobs=-2,
                          scoring='roc_auc',
                          verbose=1) 

gs_pipe_DT2.fit(Data_sample_train, target_sample_train);

Fitting 5 folds for each of 12 candidates, totalling 60 fits


[Parallel(n_jobs=-2)]: Using backend LokyBackend with 7 concurrent workers.
[Parallel(n_jobs=-2)]: Done  36 tasks      | elapsed:    3.0s
[Parallel(n_jobs=-2)]: Done  60 out of  60 | elapsed:    5.1s finished


In [98]:
gs_pipe_DT2.best_params_

{'dt__max_depth': 15, 'dt__min_samples_split': 50, 'rfi_fs__n_features_': 10}

In [99]:
gs_pipe_DT2.best_score_

0.7699379628540906

As suspected, we can achieve slightly better results with the new search space.

In [100]:
results_DT = get_search_results(gs_pipe_DT2)
results_DT.head()

KeyError: 'fsplit{i}_test_score'

We again observe that the cross-validated AUC score difference between the hyperparameter combinations is not really much. A visualization of the new search results is shown below.

In [101]:
results_DT_10_features = results_DT[results_DT['rfi_fs__n_features_'] == 10.0]

alt.Chart(results_DT_10_features, 
          title='DT Performance Comparison with 10 Features - Extended'
         ).mark_line(point=True).encode(
    alt.X('dt__min_samples_split', title='Min Samples for Split'),
    alt.Y('mean_score', title='AUC Score', scale=alt.Scale(zero=False)),
    alt.Color('dt__max_depth:N', title='Max Depth')
)

NameError: name 'results_DT' is not defined

# Performance Comparison <a class="anchor" id="5"></a> 

We have optimized each one of the three classifiers using the **training data**. We now fit the optimized models on the **test data** in a cross-validated fashion. But since cross validation itself is a random process, we perform pairwise t-tests to determine if any difference between the performance of any two optimized classifiers is statistically significant [1]. First, we perform 10-fold stratified cross-validation on each best model (without any repetitions). Second, we conduct a paired t-test for the AUC score between the following model combinations:

* KNN vs. NB,
* KNN vs. DT, and
* DT vs. NB.

[1]: For more than two classifiers, it is more appropriate to use Tukey's post-hoc tests or its non-parametric counterpart. However, these statistical inference techniques are beyond our scope.

In [ ]:
from sklearn.model_selection import cross_val_score

cv_method_ttest = StratifiedKFold(n_splits=10, random_state=111)

cv_results_KNN = cross_val_score(estimator=gs_pipe_KNN.best_estimator_,
                                 X=Data_sample_test,
                                 y=target_sample_test, 
                                 cv=cv_method_ttest, 
                                 n_jobs=-2,
                                 scoring='roc_auc')
cv_results_KNN.mean()

In [ ]:
Data_sample_test_transformed = PowerTransformer().fit_transform(Data_sample_test)

cv_results_NB = cross_val_score(estimator=gs_pipe_NB.best_estimator_,
                                X=Data_sample_test_transformed,
                                y=target_sample_test, 
                                cv=cv_method_ttest, 
                                n_jobs=-2,
                                scoring='roc_auc')
cv_results_NB.mean()

In [ ]:
cv_results_DT = cross_val_score(estimator=gs_pipe_DT2.best_estimator_,
                                X=Data_sample_test,
                                y=target_sample_test, 
                                cv=cv_method_ttest, 
                                n_jobs=-2,
                                scoring='roc_auc')
cv_results_DT.mean()

Since we fixed the same random state to be same during cross-validation, all classifiers were fitted and then tested on exactly the same test data partitions. We use the `stats.ttest_rel` function from the `SciPy` module to run the following t-tests on **test data**.

In [ ]:
from scipy import stats

print(stats.ttest_rel(cv_results_KNN, cv_results_NB))
print(stats.ttest_rel(cv_results_DT, cv_results_KNN))
print(stats.ttest_rel(cv_results_DT, cv_results_NB))

A p-value smaller than 0.05 indicates a statistically significant difference. Looking at these results, we conclude that at a 95% significance level, DT is statistically the best model in this competition (in terms of AUC) when compared on the **test data**.

Though we used AUC to optimize the algorithm hyperparameters, we shall consider the following metrics to evaluate models based on the test set:

* Accuracy
* Precision
* Recall
* F1 Score (the harmonic average of precision and recall)
* Confusion Matrix

These metrics can be computed using `classification_report` from `sklearn.metrics`. The classification reports are shown below.

In [ ]:
pred_KNN = gs_pipe_KNN.predict(Data_sample_test)

In [ ]:
Data_test_transformed = PowerTransformer().fit_transform(Data_sample_test)
pred_NB = gs_pipe_NB.predict(Data_test_transformed)

In [ ]:
pred_DT = gs_pipe_DT2.predict(Data_sample_test)

In [ ]:
from sklearn import metrics
print("\nClassification report for K-Nearest Neighbor") 
print(metrics.classification_report(target_sample_test, pred_KNN))
print("\nClassification report for Naive Bayes") 
print(metrics.classification_report(target_sample_test, pred_NB))
print("\nClassification report for Decision Tree") 
print(metrics.classification_report(target_sample_test, pred_DT))

The confusion matrices are given below.

In [ ]:
from sklearn import metrics
print("\nConfusion matrix for K-Nearest Neighbor") 
print(metrics.confusion_matrix(target_sample_test, pred_KNN))
print("\nConfusion matrix for Naive Bayes") 
print(metrics.confusion_matrix(target_sample_test, pred_NB))
print("\nConfusion matrix for Decision Tree") 
print(metrics.confusion_matrix(target_sample_test, pred_DT))

Suppose we are a tax agency and we would like to detect individuals earning more than USD 50K. Then we would choose recall as the performance metric, which is equivalent to the true positive rate (TPR). In this context, NB would be the best performer since it produces the highest recall score for incomes higher than USD 50K. The confusion matrices are in line with the classification reports. This is in contrast to our finding that DT is statistically the best performer when it comes to the AUC metric.

# Limitations and Proposed Solutions <a class="anchor" id="6"></a> 

Our modeling strategy has a few flaws and limitations. First, ours was a black-box approach since we preferred raw predictive performance over interpretability. In the future, we could consider a more in-depth analysis about the feature selection & ranking process as well as our choices for the hyperparameter spaces.

Second, we utilized a blanket power transformation on the training data when building the NB, ignoring the dummy features within the dataset. This might partially explain the poor performance of the NB when evaluated on the test set. A potential solution is to build a Gaussian NB and a Bernoulli NB separately on the numerical and dummy descriptive features respectively. Then we can compute a final prediction by multiplying predictions from each model since NB assumes inter-independence conditioned on the value of the target feature.

Third, we only worked with a small subset of the full dataset for shorter run times, both for training and testing. Since data is always valuable, we could re-run our experiments with the entire data while making sure that the train and test split is performed in a proper manner.

The DT classifier statistically outperforms the other two models. Therefore, we can perhaps improve it by further expanding the hyperparameter search space by including other parameters of this classification method. Furthermore, we can consider random forests and other ensemble methods built on trees as potentially better models.

# Summary <a class="anchor" id="7"></a> 

The Decision Tree model with 10 of the best features selected by Random Forest Importance (RFI) produces the highest cross-validated AUC score on the training data. In addition, when evaluated on the test set, the Decision Tree model again outperforms both Naive Bayes and k-Nearest Neighbor models with respect to AUC. However, the Naive Bayes model yields the highest recall score on the test data. We also observe that our models are not very sensitive to the number of features as selected by RFI when conditioned on the values of the hyperparameters in general. For this reason, it seems working with 10 features is preferable to working with the full feature set, which potentially avoids overfitting and results in models that are easier to train and easier to understand.

# References

* Jones E, Oliphant E, Peterson P, et al. SciPy: Open Source Scientific Tools for Python, 2001-, http://www.scipy.org/ [Accessed 2019-05-26].
* Lichman, M. (2013). UCI Machine Learning Repository: Census Income Data Set [online]. Available at
https://archive.ics.uci.edu/ml/datasets/adult [Accessed 2019-05-26]
* Pedregosa et al. (2011). Scikit-learn: Machine Learning in Python, Pedregosa et al., JMLR 12, pp. 2825-
2830.
* Travis E, Oliphant (2006). A guide to NumPy, USA: Trelgol Publishing.